<a href="https://colab.research.google.com/github/acastellanos-ie/natural_language_processing/blob/master/qa_practice_dl/arabic_question_answering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colab Configuration

**Execute this steps to configure the Google Colab environment in order to execute this notebook. It is not required if you are executing it locally and you have properly configured your local environment according to what explained in the Github Repository.**

The first step is to clone the repository to have access to all the data and files.

In [ ]:
! git clone https://github.com/acastellanos-ie/natural_language_processing.git

Install the requirements

In [ ]:
! pip install -Uqqr natural_language_processing/arabic_requirements.txt --use-deprecated=legacy-resolver

Ensure that you have the GPU runtime activated:

![](https://miro.medium.com/max/3006/1*vOkqNhJNl1204kOhqq59zA.png)

Now you have everything you need to execute the code in Colab

In [ ]:
#!pip install transformers==4.2.1
#!pip install pyarabic
!git clone https://github.com/aub-mind/arabert
#!pip install datasets
#!pip install sacrebleu
#!pip install fuzzysearch

     |████████████████████████████████| 1.8MB 8.0MB/s 
     |████████████████████████████████| 890kB 37.3MB/s 
     |████████████████████████████████| 2.9MB 38.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=6a462fd95e5ca1d0fd713338f446dfa6346f918649f783d528ff2ad0fea74c95
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 112kB 9.1MB/s 
  Created wheel for pyarabic: filename=PyArabic-0.6.10-cp37-none-any.whl size=113324 sha256=cddfa9d61d9244783be9595ab62630d628058e6574a3fae965e9970ca8034f3c
  Stored in directory: /root/.cache/pip/wheels/10/b8/f5/b7c1a50e6efb83544844f165a9b134afe7292585465e29b61d
Successfully built pyarabic
Cloning into 'arabert'...
remote: Enumerating objects: 240, done.
remote: Counting objects: 100% (240/240), done.
remote: Compressing objects: 100% (171/171), done.
remote: Total 454 (delt

# Imports

In [ ]:
#textwrap enables formating of long text
import textwrap

from transformers import pipeline, GPT2TokenizerFast
from arabert.aragpt2.grover.modeling_gpt2 import GPT2LMHeadModel
from arabert.preprocess import ArabertPreprocessor

#you can choose any aragpt2 model since they all have the same preprocessing
arabert_processor = ArabertPreprocessor(model_name="aragpt2-mega")

In [ ]:
import torch
device = 0 if torch.cuda.is_available() else -1
print(device)

0


# Get the Model

In [ ]:
model_name = "aubmindlab/aragpt2-mega" #the mega model needs a High-Ram colab

grover_gpt2_model = GPT2LMHeadModel.from_pretrained(model_name)
grover_gpt2_model.half()
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)

aragpt2_pipeline = pipeline("text-generation",model=grover_gpt2_model, tokenizer=tokenizer,device=device)

# Zero-Shot question answering 

Start testing some prompts, to see what works

In [ ]:
prompt = """
أجب عن السؤال التالي:
"""

In [ ]:
text = """
\"من كان رئيس ألمانيا النازية في الحرب العالمية الثانية ؟\"
"""
text_tok = tokenizer.tokenize(arabert_processor.preprocess(text))
text_len = len(text_tok)
print(text_tok)
print(text_len)

['"', 'ĠÙħÙĨ', 'ĠÙĥØ§ÙĨ', 'ĠØ±Ø¦ÙĬØ³', 'ĠØ£ÙĦÙħØ§ÙĨÙĬØ§', 'ĠØ§ÙĦÙĨØ§Ø²ÙĬØ©', 'ĠÙģÙĬ', 'ĠØ§ÙĦØŃØ±Ø¨', 'ĠØ§ÙĦØ¹Ø§ÙĦÙħÙĬØ©', 'ĠØ§ÙĦØ«Ø§ÙĨÙĬØ©', 'ĠØŁ', 'Ġ"']
12


In [ ]:
text_prep = arabert_processor.preprocess(prompt + " " + text)
input_len = len(tokenizer.tokenize(text_prep))
print(text_prep)
print(input_len)

أجب عن السؤال التالي : " من كان رئيس ألمانيا النازية في الحرب العالمية الثانية ؟ "
18


In [ ]:
print("Input Length: ", input_len + text_len //2)
gen_text = aragpt2_pipeline(text_prep,
            pad_token_id=0, # 0 for AraGPT2
            do_sample = False,
            num_beams=5,
            max_length=input_len + 10,
            top_k=10,
            top_p=0.95,
            repetition_penalty = 3.0,
            no_repeat_ngram_size = 3,
            num_return_sequences = 5)
gen_text

Input Length:  24


[{'generated_text': 'أجب عن السؤال التالي : " من كان رئيس ألمانيا النازية في الحرب العالمية الثانية ؟ " الجواب هو أدولف هتلر ، الذي حكم ألمانيا بين'},
 {'generated_text': 'أجب عن السؤال التالي : " من كان رئيس ألمانيا النازية في الحرب العالمية الثانية ؟ " الجواب هو أدولف هتلر ، الذي حكم ألمانيا لمدة'},
 {'generated_text': 'أجب عن السؤال التالي : " من كان رئيس ألمانيا النازية في الحرب العالمية الثانية ؟ " الجواب هو أدولف هتلر ، الذي ولد في النمسا'},
 {'generated_text': 'أجب عن السؤال التالي : " من كان رئيس ألمانيا النازية في الحرب العالمية الثانية ؟ " الجواب هو أدولف هتلر ، الذي حكم ألمانيا منذ'},
 {'generated_text': 'أجب عن السؤال التالي : " من كان رئيس ألمانيا النازية في الحرب العالمية الثانية ؟ " إن الإجابة على هذا السؤال صعبة جدا ، لأن هتلر'}]

In [ ]:
text_prep = arabert_processor.preprocess("أجب عن السؤال التالي : في أي سنة تأسست مدينة العيون ؟ الجواب هو في سنة")
input_len = len(tokenizer.tokenize(text_prep))
print(text_prep)
print(input_len)

أجب عن السؤال التالي : في أي سنة تأسست مدينة العيون ؟ الجواب هو في سنة
17


In [ ]:
print("Input Length: ", input_len + text_len //2)
gen_text = aragpt2_pipeline("أجب عن السؤال التالي : في أي سنة تأسست مدينة العيون ؟ الجواب هو في سنة",
            pad_token_id=0, # 0 for AraGPT2
            do_sample = False,
            num_beams=5,
            max_length=input_len + 4,
            top_k=10,
            top_p=0.95,
            repetition_penalty = 3.0,
            no_repeat_ngram_size = 3,
            num_return_sequences = 5)
gen_text

Input Length:  23


[{'generated_text': 'أجب عن السؤال التالي : في أي سنة تأسست مدينة العيون ؟ الجواب هو في سنة 1909 �'},
 {'generated_text': 'أجب عن السؤال التالي : في أي سنة تأسست مدينة العيون ؟ الجواب هو في سنة 1906 �'},
 {'generated_text': 'أجب عن السؤال التالي : في أي سنة تأسست مدينة العيون ؟ الجواب هو في سنة 1902 �'},
 {'generated_text': 'أجب عن السؤال التالي : في أي سنة تأسست مدينة العيون ؟ الجواب هو في سنة 1904 �'},
 {'generated_text': 'أجب عن السؤال التالي : في أي سنة تأسست مدينة العيون ؟ الجواب هو في سنة 1903 �'}]